In [2]:
from pre_process import PreProcess
from word_to_vector import WordToVector
from one_hot_encoding import OneHotEncoder
import numpy as np
from training import DnnTraining
from inference import DnnInference
from keras.models import load_model

In [3]:
word_arr_one, label_arr_one = PreProcess.getTrainingTuple(dataFile='../../data/en-train.conll', onlyBioTagging=True)
word_arr_two, label_arr_two = PreProcess.getTrainingTuple(dataFile='../../data/en-dev.conll', onlyBioTagging=True)

word_arr = word_arr_one + word_arr_two
label_arr = label_arr_one + label_arr_two

n = len(word_arr)
train_split = int(0.7 * n)
val_split = int(0.2 * n)
train_word_arr = word_arr[:train_split]
val_word_arr = word_arr[train_split:train_split + val_split]
test_word_arr = word_arr[train_split + val_split:]
print(len(train_word_arr), len(val_word_arr), len(test_word_arr))

m = len(label_arr)
train_label_split = int(0.7 * m)
val_label_split = int(0.2 * m)
train_label_arr = label_arr[:train_label_split]
val_label_arr = label_arr[train_label_split:train_label_split + val_label_split]
test_label_arr = label_arr[train_label_split + val_label_split:]
print(len(train_label_arr), len(val_label_arr), len(test_label_arr))


186433 53266 26635
186433 53266 26635


In [5]:
# Get wordToVector from [wordArr] and oneHotEncoding from [labelArr]
wordToVecArr = WordToVector.getPretrainedWordToVecList(train_word_arr)
oneHotEncodingArr = OneHotEncoder.getOneHotEncodingOfOutput(train_label_arr)
# Convert python array to num py array
np_wordToVecArr = np.array(wordToVecArr)
np_oneHotEncodingArr = np.array(oneHotEncodingArr)
print("Train set preprocess done")

# Get wordToVector from [wordArr] and oneHotEncoding from [labelArr]
val_wordToVecArr = WordToVector.getPretrainedWordToVecList(val_word_arr)
val_oneHotEncodingArr = OneHotEncoder.getOneHotEncodingOfOutput(val_label_arr)
# Convert python array to num py array
val_np_wordToVecArr = np.array(val_wordToVecArr)
val_np_oneHotEncodingArr = np.array(val_oneHotEncodingArr)
print("Validation set preprocess done")

training = DnnTraining(input_dim=300, output_dim=3)
training.startTraining(np_wordToVecArr, np_oneHotEncodingArr, val_np_wordToVecArr, val_np_oneHotEncodingArr, epochs=100)
training.saveTrainedModel()


Train set preprocess done
Validation set preprocess done
Epoch 1/100
5827/5827 [==============================] - 5s 817us/step - loss: 0.4332 - accuracy: 0.8391 - f1_score: 0.8354 - val_loss: 0.4436 - val_accuracy: 0.8327 - val_f1_score: 0.8249
Epoch 2/100
5827/5827 [==============================] - 5s 789us/step - loss: 0.3925 - accuracy: 0.8497 - f1_score: 0.8468 - val_loss: 0.4385 - val_accuracy: 0.8353 - val_f1_score: 0.8288
Epoch 3/100
5827/5827 [==============================] - 5s 793us/step - loss: 0.3844 - accuracy: 0.8520 - f1_score: 0.8503 - val_loss: 0.4344 - val_accuracy: 0.8349 - val_f1_score: 0.8310
Epoch 4/100
5827/5827 [==============================] - 5s 815us/step - loss: 0.3813 - accuracy: 0.8545 - f1_score: 0.8514 - val_loss: 0.4325 - val_accuracy: 0.8348 - val_f1_score: 0.8306
Epoch 5/100
5827/5827 [==============================] - 5s 776us/step - loss: 0.3766 - accuracy: 0.8552 - f1_score: 0.8532 - val_loss: 0.4347 - val_accuracy: 0.8356 - val_f1_score: 0.830

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from keras import backend as K

model = training.getCurrentModel()

# Get wordToVector from [TestWordArr] and oneHotEncoding from [TestLabelArr]
test_wordToVecArr = WordToVector.getPretrainedWordToVecList(test_word_arr)
test_oneHotEncodingArr = OneHotEncoder.getOneHotEncodingOfOutput(test_label_arr)
# Convert python array to num py array
test_np_wordToVecArr = np.array(test_wordToVecArr)
test_np_oneHotEncodingArr = np.array(test_oneHotEncodingArr)

# Assuming you have test data and labels
x_test = test_np_wordToVecArr
y_test = test_np_oneHotEncodingArr


In [11]:
# Perform inference on the test set
y_pred = model.predict(x_test)

y_test_argmax = np.argmax(y_test, axis=1)

# Convert predicted probabilities to class labels (if needed)
y_pred_classes = np.argmax(y_pred, axis=1)

def internal_f1_score(y_true, y_pred):
    """
    Custom F1 score metric function.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))

    precision = true_positives / (predicted_positives)
    recall = true_positives / (possible_positives)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

# Compute evaluation metrics
accuracy = accuracy_score(y_test_argmax, y_pred_classes)
f1_internal = internal_f1_score(y_test_argmax, y_pred_classes)

print('Accuracy:', accuracy)
print('F1 score internal:', f1_internal.numpy())

833/833 [==============================] - 0s 376us/step
Accuracy: 0.8392716350666416
F1 score internal: 0.9547423389871676
